In [16]:
import pandas as pd
import numpy as np

In [17]:
data = pd.read_parquet('./data/train.parquet')

In [18]:
NAN_VALUE = -127 
data = data.fillna(NAN_VALUE) 

In [19]:
data.shape

(5531451, 190)

In [20]:
data['customer_ID'].value_counts().reset_index()['customer_ID'].value_counts()

13    386034
12     10623
10      6721
9       6411
8       6110
2       6098
11      5961
3       5778
6       5515
7       5198
1       5120
4       4673
5       4671
Name: customer_ID, dtype: int64

In [21]:
data_customer_13 = data.loc[data.groupby('customer_ID')['customer_ID'].transform(len) == 13]
data_customer_not_13 = data.loc[data.groupby('customer_ID')['customer_ID'].transform(len) != 13]

In [22]:
customer_IDs = data_customer_13['customer_ID'].unique()
train_ids_size = int(len(customer_IDs) * 0.67)
valid_ids_size = int(len(customer_IDs) * 0.03)
other_ids_size = int(len(customer_IDs) * 0.3)

selection = [train_ids_size, valid_ids_size, other_ids_size]

customer_ids_split = np.split(customer_IDs, np.cumsum(selection))

train_ids = customer_ids_split[0]
valid_ids = customer_ids_split[1]
other_ids = customer_ids_split[2]


In [23]:
train_df = data_customer_13[data_customer_13['customer_ID'].isin(train_ids)]
valid_df = data_customer_13[data_customer_13['customer_ID'].isin(valid_ids)]
other_df = data_customer_13[data_customer_13['customer_ID'].isin(other_ids)]

train_combined_df = pd.concat([train_df, data_customer_not_13], ignore_index=True)


In [24]:
train_df.shape, valid_df.shape, other_df.shape, train_combined_df.shape

((3362346, 190), (150553, 190), (1505530, 190), (3875355, 190))

In [25]:
train_combined_df.to_parquet('./data/train_raw.parquet', index=False)
valid_df.to_parquet('./data/valid_raw.parquet', index=False)
other_df.to_parquet('./data/other_raw.parquet', index=False)


In [26]:
importance_fea = ['P_2','D_39','D_42','D_43','D_46','D_47','B_1','B_3','B_4','B_5','B_9','B_11','B_14','S_3','S_12','R_1'] 

importance_fea_column = ['customer_ID','S_2'] + importance_fea

train_combined_fea_df = train_combined_df[importance_fea_column]
valid_fea_df = valid_df[importance_fea_column]
other_fea_df = other_df[importance_fea_column]

In [27]:
train_combined_fea_df.to_parquet('./data/train_importance_fea.parquet', index=False)
valid_fea_df.to_parquet('./data/valid_importance_fea.parquet', index=False)
other_fea_df.to_parquet('./data/other_importance_fea.parquet', index=False)
